In [32]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import date, timedelta
from config import password

In [33]:
#importing all csv datasets
ny_une = "csv/unemploymentnycounty.csv"
cal_une = "csv/unemploymentcalcounty.csv"
fl_une = "csv/FLunemploymentcountyjune.csv"

In [34]:
ny_df = pd.read_csv(ny_une)
fl_df = pd.read_csv(fl_une)
cal_df = pd.read_csv(cal_une)

In [41]:
ny_df2 = ny_df.rename(columns = {"RANK" : 'Unemployment Rank', "COUNTY":  'County', "RATE": 'Unemployment_Rate'})
ny_df2 = ny_df2.drop(columns = ['Unemployment Rank'])
ny_df2['Year'] = "2020"
ny_df2['Month'] = "June"
ny_df2["State"] = "NY"
ny_df2.reset_index()
ny_final = ny_df2.set_index('Month')
ny_final.dropna()
ny_final.to_csv('ny_final2.csv')

In [40]:
fl_df.head()
fl_df2 = fl_df.drop(['LABOR', 'EMPLOY-','UNEMPLOYMENT'], axis=1)
fl_df3 = fl_df2.rename(columns = {"Unnamed: 0": "County", "Unnamed: 4": "Unemployment_Rate"})
fl_df_final = fl_df3.drop([0, 0])
fl_df_final.reset_index()
fl_df_final["Year"] = "2020"
fl_df_final["Month"] = "June"
fl_df_final["State"] = "FL"
fl_final = fl_df_final.set_index('Month')
fl_final.dropna()
fl_final.to_csv('fl_final2.csv')

In [39]:
cal_df.head()
cal_df2 = cal_df.drop(columns = ['Area Type ','Seasonally Adjusted (Y/N) ','Status (Preliminary / Final) ','Unemployment ','Employment ','Labor Force'])
cal_df2['Date'] = pd.to_datetime(cal_df2['Date'])

res = cal_df2[~(cal_df2['Date'] <= '2020-05-31')]

cal_df_final = res
cal_df_final
cal_tidy = cal_df_final.rename(columns = {'Area Name ': 'County_CA', 'Unemployment Rate': 'Unemployment_Rate'}, inplace = False)
cal_final= cal_tidy[cal_tidy.County_CA != 'California']
cal_final = cal_final.rename(columns = {'County_CA': 'County, CA'}, inplace = False)
# cal_final.reset_index()
cal_final = cal_final.drop(columns = ['Date'])
cal_final["County, CA"]= cal_final["County, CA"].str.split(",", n = 1, expand = True)
cal_final["State"] = "CA"
cal_final = cal_final.rename(columns = {'County, CA':  'County'}, inplace= False)
cal_final = cal_final.set_index('Month')
cal_final = cal_final.rename(columns = {'Unemployment Rate ': 'Unemployment_Rate'})

cal_final.dropna()
cal_final.to_csv('cal_final2.csv')

,County,Year,Unemployment_Rate,State
Month,,,,
June,Alameda County,2020,0.135,CA
June,Alpine County,2020,0.189,CA
June,Amador County,2020,0.127,CA
June,Butte County,2020,0.122,CA
June,Calaveras County,2020,0.110,CA
...,...,...,...,...
June,Santa Rosa,2020,0.114,CA
June,Stockton-Lodi,2020,0.159,CA
June,Vallejo-Fairfield,2020,0.136,CA


In [18]:
# unemployment_df = [cal_final,ny_df2, fl_df_final]
# result_1 = pd.merge(unemployment_df, join='inner', axis=1)
final = pd.merge(cal_final,fl_final, on ='Month', how='outer')
# df = pd.DataFrame(result_1)
# df.dropna(axis=1, how='all')
unemployment_df = final.merge(ny_final, on = 'Month', how='outer')

unemployment_df.head()

,County,Year,Unemployment Rate,State_x,"County, FL",Unemployment Rate_x,Year_x,State_y,"County, NY",Unemployment Rate_y,Year_y,State
Month,,,,,,,,,,,,
June,Alameda County,2020,0.135,CA,Alachua County,6.5%,2020,FL,Hamilton County,7.7,2020,NY
June,Alameda County,2020,0.135,CA,Alachua County,6.5%,2020,FL,Chenango County,8.4,2020,NY
June,Alameda County,2020,0.135,CA,Alachua County,6.5%,2020,FL,Yates County,8.5,2020,NY
June,Alameda County,2020,0.135,CA,Alachua County,6.5%,2020,FL,Columbia County,8.8,2020,NY
June,Alameda County,2020,0.135,CA,Alachua County,6.5%,2020,FL,Tompkins County,8.9,2020,NY


In [19]:
#connect to postgres
engine = create_engine(password)


In [20]:
engine.table_names()

[]